In [2]:
from typing import List, Tuple, Dict
import cv2
import numpy as np
import os
from roboflow import Roboflow
from tqdm import tqdm
import random
import matplotlib.pyplot as plt 
from PIL import Image

def convert_to_pixels(box: List[float], image_width: int, image_height: int) -> Tuple[int, int, int, int]:
    x_center, y_center, width, height = box
    x_center *= image_width
    y_center *= image_height
    width *= image_width
    height *= image_height
    
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)
    
    return x1, y1, x2, y2

def crop_and_save(image_path, bbox_coords, output_dir, class_id):
    """
    Crop the image to the bounding box and save it to the specified output directory.
    """
    with Image.open(image_path) as img:
        cropped_img = img.crop(bbox_coords)
        output_path = os.path.join(output_dir, f"{class_id}", os.path.basename(image_path))
        cropped_img.save(output_path)

def create_classification_dataset(base_dir, output_dir):
    images_dir = os.path.join(base_dir, "images")
    labels_dir = os.path.join(base_dir, "labels")
    os.makedirs(output_dir, exist_ok=True)
    for i in range(2):  # Number of classes
        os.makedirs(os.path.join(output_dir, f"{i}"), exist_ok=True)

    # Process each label file
    for label_file in tqdm(os.listdir(labels_dir)):
        with open(os.path.join(labels_dir, label_file), 'r') as file:
            lines = file.readlines()

        if not lines:
            continue

        # Check for class "1" first, then fall back to class "0"
        selected_line = None
        for line in lines:
            class_id, _ = line.strip().split(' ', 1)
            if class_id == '1':
                selected_line = line
                break
            elif class_id == '0' and selected_line is None:
                selected_line = line

        if selected_line is None:
            continue  # Skip if no suitable class found

        class_id, coords = selected_line.strip().split(' ', 1)
        box = [float(coord) for coord in coords.split()]
        image_file = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_dir, image_file)

        if not os.path.exists(image_path):
            continue

        img = Image.open(image_path)
        bbox_coords = convert_to_pixels(box, img.width, img.height)
        crop_and_save(image_path, bbox_coords, output_dir, class_id)




In [3]:
# Directories
base_dir = "detection_dataset_groupby_env"
output_dir = "classification_dataset_groupby_env"
all_env_dirs = [os.path.join(base_dir, env_dir) for env_dir in os.listdir(base_dir)]
print(all_env_dirs)

for env_dir in all_env_dirs:
    output_dir_env = os.path.join(output_dir, env_dir.split("/")[-1])
    create_classification_dataset(env_dir, output_dir_env)

['detection_dataset_groupby_env/other', 'detection_dataset_groupby_env/mima', 'detection_dataset_groupby_env/group_3', 'detection_dataset_groupby_env/philipine_night', 'detection_dataset_groupby_env/group_2', 'detection_dataset_groupby_env/group_1', 'detection_dataset_groupby_env/korean_outdoor', 'detection_dataset_groupby_env/korean_organized', 'detection_dataset_groupby_env/group_4']


100%|██████████| 7/7 [00:00<00:00, 563.29it/s]


In [2]:
def find_small_images(directory, min_width, min_height = None):
    print(directory)
    min_height = min_width if not min_height else min_height
    small_images_info = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # Skip system files like .DS_Store
            if filename.startswith('.'):
                continue
            try:
                file_path = os.path.join(dirpath, filename)
                with Image.open(file_path) as img:
                    w, h = img.size
                    if w * h < min_width * min_height:
                        small_images_info.append((file_path, img.size))
            except Exception as e:
                print(f"Error processing image {filename}: {e}")
    return small_images_info

small_images_info = find_small_images(output_dir + "/0", 17)


classification_dataset/0


In [4]:
for img in small_images_info:
    os.remove(img[0])

# Create new cls dataset with new env


In [76]:
# import packages to split dataset into train and test and val
from sklearn.model_selection import train_test_split
import os
import numpy as np
from tqdm import tqdm   
import shutil

In [87]:
root_dir = "classification_dataset_groupby_env"
envs = os.listdir(root_dir)
envs.remove("other")
# chekc length of each env

envs_shuffled = np.random.permutation(envs)
envs_train, envs_val = train_test_split(envs_shuffled, test_size=0.3)
envs_val, envs_test = train_test_split(envs_val, test_size=0.3)

print(len(envs_train), len(envs_val), len(envs_test))

5 2 1


In [88]:
select_n_from_env = 20
env_train_pos = []
env_val_pos = []
env_test_pos = []
env_train_neg = []
env_val_neg = []
env_test_neg = []

for env in list(envs_train) + list(envs_val) + list(envs_test):
    env_dir = os.path.join(root_dir, env)
    pos = [os.path.join(env_dir, "1", img) for img in os.listdir(os.path.join(env_dir, "1"))]
    neg = [os.path.join(env_dir, "0", img) for img in os.listdir(os.path.join(env_dir, "0"))]
    
    n_pos = select_n_from_env if len(pos) >= select_n_from_env else len(pos)
    n_neg = select_n_from_env if len(neg) >= select_n_from_env else len(neg)
    
    
    
    selected_pos = np.random.choice(pos, n_pos, replace=False).tolist()
    selected_neg = np.random.choice(neg, n_neg, replace=False)
    
    if env in envs_train:
        env_train_pos.extend(selected_pos)
        env_train_neg.extend(selected_neg)
    elif env in envs_val:
        env_val_pos.extend(selected_pos)
        env_val_neg.extend(selected_neg)
    elif env in envs_test:
        env_test_pos.extend(selected_pos)
        env_test_neg.extend(selected_neg)


In [89]:
other_images = os.listdir(os.path.join(root_dir, "other"))
other_images_pos = os.listdir(os.path.join(root_dir, "other", "1"))
other_images_neg = os.listdir(os.path.join(root_dir, "other", "0"))

In [91]:
other_images_pos = [os.path.join(root_dir, "other", "1", img) for img in other_images_pos]
other_images_neg = [os.path.join(root_dir, "other", "0", img) for img in other_images_neg]

other_train_pos, other_val_pos = train_test_split(other_images_pos, test_size=0.3)
other_val_pos, other_test_pos = train_test_split(other_val_pos, test_size=0.33)

other_train_neg, other_val_neg = train_test_split(other_images_neg, test_size=0.3)
other_val_neg, other_test_neg = train_test_split(other_val_neg, test_size=0.33)

all_train_pos = env_train_pos + other_train_pos
all_val_pos = env_val_pos + other_val_pos
all_test_pos = env_test_pos + other_test_pos

all_train_neg = env_train_neg + other_train_neg
all_val_neg = env_val_neg + other_val_neg
all_test_neg = env_test_neg + other_test_neg

In [93]:
print(len(all_train_pos), len(all_val_pos), len(all_test_pos))
print(len(all_train_neg), len(all_val_neg), len(all_test_neg))

145 57 15
770 212 116


In [94]:
keep_p = 0.5
# apply keep_p to all negative images
all_train_neg = np.random.choice(all_train_neg, int(len(all_train_neg) * keep_p), replace=False)
all_val_neg = np.random.choice(all_val_neg, int(len(all_val_neg) * keep_p), replace=False)
all_test_neg = np.random.choice(all_test_neg, int(len(all_test_neg) * keep_p), replace=False)

print(len(all_train_neg), len(all_val_neg), len(all_test_neg))

385 106 58


In [95]:
new_dataset_dir = "classification_dataset_groupby_env_split"
os.makedirs(new_dataset_dir, exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "test"), exist_ok=True)

os.makedirs(os.path.join(new_dataset_dir, "train", "1"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "val", "1"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "test", "1"), exist_ok=True)

os.makedirs(os.path.join(new_dataset_dir, "train", "0"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "val", "0"), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, "test", "0"), exist_ok=True)

for img in tqdm(all_train_pos):
    shutil.copy(img, os.path.join(new_dataset_dir, "train", "1"))
for img in tqdm(all_val_pos):
    shutil.copy(img, os.path.join(new_dataset_dir, "val", "1", os.path.basename(img)))
for img in tqdm(all_test_pos):
    shutil.copy(img, os.path.join(new_dataset_dir, "test", "1", os.path.basename(img)))
    
for img in tqdm(all_train_neg):
    shutil.copy(img, os.path.join(new_dataset_dir, "train", "0", os.path.basename(img)))
for img in tqdm(all_val_neg):
    shutil.copy(img, os.path.join(new_dataset_dir, "val", "0", os.path.basename(img)))
for img in tqdm(all_test_neg):
    shutil.copy(img, os.path.join(new_dataset_dir, "test", "0", os.path.basename(img)))
    
    
    
    
    

100%|██████████| 58/58 [00:00<00:00, 2305.39it/s]


In [84]:
img

'detection_dataset_groupby_env/other/1/classification_dataset_groupby_env/other/1/classification_dataset_groupby_env/other/1/TUHMXNRF2TNJ_jpg.rf.614b987a47ab2f02965abb542d0487b6.jpg'

In [85]:
all_train_pos

['classification_dataset_groupby_env/mima/1/Screenshot-2023-07-24-at-2-15-38-PM-2-_png.rf.7175f96f89261f0fe6d2945b89752d88.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-24-at-2-14-42-PM-2-_png.rf.8a0233b282122a3a6f95aadeff842717.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-24-at-12-51-36-PM-2-_png.rf.aeeeac7eb932aceef7865fc6c00e0b49.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-01-at-1-30-43-AM-2-_png.rf.0ff2d07edfe39028b6cb2e43709d1e57.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-02-at-4-19-39-PM-2-_png.rf.788a37870dd9d6cd1652dda9bd48afda.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-24-at-2-14-22-PM-2-_png.rf.e3018d16be5c3c41486635edf3ebae5b.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-24-at-12-49-44-PM-2-_png.rf.abefe62afad54fb8be3231f8ecd0acc8.jpg',
 'classification_dataset_groupby_env/mima/1/Screenshot-2023-07-02-at-4-20-35-PM-2-_png.rf.e8564